In [1]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -q

In [2]:
# third party

# syft absolute
import syft as sy

sy.requires(SYFT_VERSION)

✅ The installed version of syft==0.8.7b13 matches the requirement >=0.8.2b0 and the requirement <0.9


### Launch a Syft Domain Server

In [3]:
# Launch a fresh domain server named "test-domain-1" in dev mode on the local machine
node = sy.orchestra.launch(name="test-domain-1", dev_mode=True, reset=True)

SyftInfo: You have launched a development node at http://0.0.0.0:None.It is intended only for local use.

In [4]:
# list(node.python_node.get_service("userservice").stash.partition.data.values())[0].verify_key

In [5]:
# log into the node with default root credentials
client = node.login(email="info@openmined.org", password="changethis")

Logged into <test-domain-1: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [6]:
# stdlib
from typing import Any

In [7]:
def helper_make_action_obj(orig_obj: Any):
    return sy.ActionObject.from_obj(orig_obj)


def helper_make_action_pointers(worker, obj, *args, **kwargs):
    root_domain_client = worker
    res = obj.send(root_domain_client)
    obj_pointer = root_domain_client.api.services.action.get_pointer(res.id)

    # The args and kwargs should automatically be pointerized by obj_pointer
    return obj_pointer, args, kwargs

In [8]:
assert client.settings.enable_eager_execution(enable=True)

orig_obj, op, args, kwargs, expected = (1, "__add__", [1], {}, 2)

obj = helper_make_action_obj(orig_obj)

obj_pointer, args_pointers, kwargs_pointers = helper_make_action_pointers(
    client, obj, *args, **kwargs
)

In [9]:
# syft absolute
from syft import UID
from syft.service.action.action_object import ActionType
from syft.service.action.action_object import PreHookContext

In [10]:
obj_pointer.syft_point_to(client.id)

context = PreHookContext(obj=obj_pointer, op_name=op, action_type=ActionType.METHOD)

context, result_args, result_kwargs = obj_pointer._syft_run_pre_hooks__(
    context, name=op, args=args_pointers, kwargs=kwargs_pointers
)

> /Users/koen/workspace/PySyft/packages/syft/src/syft/service/action/action_service.py(794)execute()
    793 
--> 794         return set_result
    795 



ipdb>  c


> /Users/koen/workspace/PySyft/packages/syft/src/syft/service/action/action_service.py(794)execute()
    793 
--> 794         return set_result
    795 



ipdb>  c


In [11]:
result_kwargs

{}

In [12]:
assert context.result_id is not None

context.obj.syft_node_uid = UID()
result = obj_pointer._syft_run_post_hooks__(context, name=op, result=obj_pointer)
assert result.syft_node_uid == context.obj.syft_node_uid